<a href="https://colab.research.google.com/github/AsmaDamani/PySpark/blob/master/Deposit%20Attrition%20Query.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyspark

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession \
 .builder \
 .appName("Python Spark SQL basic example") \
 .config("spark.some.config.option", "some-value") \
 .getOrCreate()

In [11]:
data = spark.read.csv('refine_zone_table_sample.csv',inferSchema= True, header = True)

In [12]:
data.show()

+----+------+----+------+-----+------+------+------+------+------+------+------+------+------+-------+-------+------+-------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+----------+
|srab|  sran|sras|srsbtp|srsmo|srbp01|srbp02|srbp03|srbp04|srbp05|srbp06|srbp07|srbp08|srbp09| srbp10| srbp11|srbp12| srbptd|srbpml|srnd01|srnd02|srnd03|srnd04|srnd05|srnd06|srnd07|srnd08|srnd09|srnd10|srnd11|srnd12|srndtd|srndml|etl_id|  eod_date|
+----+------+----+------+-----+------+------+------+------+------+------+------+------+------+-------+-------+------+-------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+----------+
|2389|CILJ4M|   1|     D|    P|     0|     0|     0|     0|     0|     0|     0|     0|     0|      0|      0|     0|      0|     0|     0|     0|     0|     0|     0|     0|     0|     0|     0|     0|     0|     0|     0|     0|     1|11/30/2020|
|238

In [13]:
data.registerTempTable("data")

In [14]:
from pyspark.sql import SQLContext
sc = spark.sparkContext
sqlContext = SQLContext(sc)

In [15]:
sqlContext.sql('select * from data').show()

+----+------+----+------+-----+------+------+------+------+------+------+------+------+------+-------+-------+------+-------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+----------+
|srab|  sran|sras|srsbtp|srsmo|srbp01|srbp02|srbp03|srbp04|srbp05|srbp06|srbp07|srbp08|srbp09| srbp10| srbp11|srbp12| srbptd|srbpml|srnd01|srnd02|srnd03|srnd04|srnd05|srnd06|srnd07|srnd08|srnd09|srnd10|srnd11|srnd12|srndtd|srndml|etl_id|  eod_date|
+----+------+----+------+-----+------+------+------+------+------+------+------+------+------+-------+-------+------+-------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+----------+
|2389|CILJ4M|   1|     D|    P|     0|     0|     0|     0|     0|     0|     0|     0|     0|      0|      0|     0|      0|     0|     0|     0|     0|     0|     0|     0|     0|     0|     0|     0|     0|     0|     0|     0|     1|11/30/2020|
|238

In [21]:
sqlContext.sql('SELECT d.*, ROW_NUMBER() OVER (PARTITION BY srab,srsbtp ORDER BY eod_date DESC) AS rn\
  FROM data AS d').show()

+----+------+----+------+-----+------+------+------+------+------+------+------+------+------+-------+-------+------+-------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+----------+---+
|srab|  sran|sras|srsbtp|srsmo|srbp01|srbp02|srbp03|srbp04|srbp05|srbp06|srbp07|srbp08|srbp09| srbp10| srbp11|srbp12| srbptd|srbpml|srnd01|srnd02|srnd03|srnd04|srnd05|srnd06|srnd07|srnd08|srnd09|srnd10|srnd11|srnd12|srndtd|srndml|etl_id|  eod_date| rn|
+----+------+----+------+-----+------+------+------+------+------+------+------+------+------+-------+-------+------+-------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+----------+---+
|2389|CILJ4M|   1|     D|    P|     0|     0|     0|     0|     0|     0|     0|     0|     0|      0|      0|     0|      0|     0|     0|     0|     0|     0|     0|     0|     0|     0|     0|     0|     0|     0|     0|     0|     1| 12/

In [22]:
sqlContext.sql('WITH ranked_messages AS (\
  SELECT d.*, ROW_NUMBER() OVER (PARTITION BY srab,srsbtp ORDER BY eod_date DESC) AS rn\
  FROM data AS d\
)\
SELECT srab, srsbtp, srbp01, srbp02, srbp03, srbp04, srbp05, srbp06, srbp07, srbp08, srbp09, srbp10, srbp11, srbp12 \
srnd01, srnd02, srnd03, srnd04, srnd05, srnd06, srnd07, srnd08, srnd09, srnd10, srnd11, srnd12 \
 FROM ranked_messages WHERE rn = 1').show()

+----+------+------+------+------+------+------+------+------+------+------+-------+-------+------+------+------+------+------+------+------+------+------+------+------+------+
|srab|srsbtp|srbp01|srbp02|srbp03|srbp04|srbp05|srbp06|srbp07|srbp08|srbp09| srbp10| srbp11|srnd01|srnd02|srnd03|srnd04|srnd05|srnd06|srnd07|srnd08|srnd09|srnd10|srnd11|srnd12|
+----+------+------+------+------+------+------+------+------+------+------+-------+-------+------+------+------+------+------+------+------+------+------+------+------+------+
|2389|     D|     0|     0|     0|     0|     0|     0|     0|     0|     0|      0|      0|     0|     0|     0|     0|     0|     0|     0|     0|     0|     0|     0|     0|
|2389|     C|     0|     0|     0|     0|     0|     0|     0|     0|     0|1694900|4704600|     0|     0|     0|     0|     0|     0|     0|     0|     0|    31|    30|     0|
+----+------+------+------+------+------+------+------+------+------+------+-------+-------+------+------+------+--

In [27]:
sqlContext.sql('WITH ranked_messages AS (\
  SELECT d.*, ROW_NUMBER() OVER (PARTITION BY srab,srsbtp ORDER BY eod_date DESC) AS rn\
  FROM data AS d\
)\
SELECT srab, srsbtp, srbp01/srnd01 as avgm1, srbp02/srnd02 as avgm2, srbp03/srnd03 as avgm3, srbp04/srnd04 as avgm4, srbp05/srnd05 as avgm4, srbp06/srnd06 as avgm6, \
srbp07/srnd07 as avgm7, srbp08/srnd08 as avgm8, srbp09/srnd09 as avgm9, srbp10/srnd10 as avgm10, srbp11/srnd11 as avgm11, srbp12/srnd12 as avgm12 \
 FROM ranked_messages WHERE rn = 1').show()

+----+------+-----+-----+-----+-----+-----+-----+-----+-----+-----+----------------+--------+------+
|srab|srsbtp|avgm1|avgm2|avgm3|avgm4|avgm4|avgm6|avgm7|avgm8|avgm9|          avgm10|  avgm11|avgm12|
+----+------+-----+-----+-----+-----+-----+-----+-----+-----+-----+----------------+--------+------+
|2389|     D| null| null| null| null| null| null| null| null| null|            null|    null|  null|
|2389|     C| null| null| null| null| null| null| null| null| null|54674.1935483871|156820.0|  null|
+----+------+-----+-----+-----+-----+-----+-----+-----+-----+-----+----------------+--------+------+



In [29]:
df = sqlContext.sql('WITH ranked_messages AS (\
  SELECT d.*, ROW_NUMBER() OVER (PARTITION BY srab,srsbtp ORDER BY eod_date DESC) AS rn\
  FROM data AS d\
)\
SELECT srab, srsbtp, srbp01/srnd01 as avgm1, srbp02/srnd02 as avgm2, srbp03/srnd03 as avgm3, srbp04/srnd04 as avgm4, srbp05/srnd05 as avgm4, srbp06/srnd06 as avgm6, \
srbp07/srnd07 as avgm7, srbp08/srnd08 as avgm8, srbp09/srnd09 as avgm9, srbp10/srnd10 as avgm10, srbp11/srnd11 as avgm11, srbp12/srnd12 as avgm12 \
 FROM ranked_messages WHERE rn = 1').persist()

In [30]:
df.show()

+----+------+-----+-----+-----+-----+-----+-----+-----+-----+-----+----------------+--------+------+
|srab|srsbtp|avgm1|avgm2|avgm3|avgm4|avgm4|avgm6|avgm7|avgm8|avgm9|          avgm10|  avgm11|avgm12|
+----+------+-----+-----+-----+-----+-----+-----+-----+-----+-----+----------------+--------+------+
|2389|     D| null| null| null| null| null| null| null| null| null|            null|    null|  null|
|2389|     C| null| null| null| null| null| null| null| null| null|54674.1935483871|156820.0|  null|
+----+------+-----+-----+-----+-----+-----+-----+-----+-----+-----+----------------+--------+------+



In [35]:
from pyspark.sql.functions import *
from pyspark.sql import *
from pyspark.sql.types import *

In [36]:
df.groupBy().pivot("srsbtp").agg(first("avg10")).show()


AnalysisException: ignored

In [ ]:
m1 = '10'
m2 = '11'

In [28]:
sqlContext.sql('select srab, avgm10, avgm11 from data').show()

AnalysisException: ignored